##### Підготовка та Аналіз даних

Лабораторна робота №2

Наука про дані: підготовчий етап

Мета роботи: ознайомитися з основними кроками по роботі з даними – workflow
від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі
та природу даних, над якими виконується аналітичні операції

Основні поняття: сирі дані (raw data), підготовка даних (data preparation)

---

ФБ-35

**Прилуцький Олександр**

Імпорт бібліотек для подальшої роботи
---

In [1]:
from datetime import datetime
import requests
import urllib.request
import os
import pandas as pd
from IPython.display import display
import math
print("Setup done")

Setup done


Завантаження даних
---

Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура має бути
автоматизована, параметром процедури має бути індекс (номер) області. При
зберіганні файлу до його імені потрібно додати дату та час завантаження.

Передбачити повторні запуски скрипту, довантаження нових даних та колізію
даних;

In [2]:
def download_function(reg_id):
    url = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID=" + str(reg_id) + "&year1=1981&year2=2020&type=Mean"
    timestamp = datetime.now()
    timestamp = timestamp.strftime("%Y-%m-%d_%H-%M-%S")
    folder = "CSVs"
    
    filename = os.path.join(folder, f"data_region_{reg_id}_{timestamp}.csv")
    try:
        recieved_data = requests.get(url, timeout=3)
        recieved_data.raise_for_status()
        with open(filename, "w", encoding="utf-8") as file:
            file.write(recieved_data.text)
        print(f"Saving region {reg_id} in {filename} [OK]")
    except:
        print(f"Error in region {reg_id}")


if __name__ == "__main__":
    folder = "CSVs"
    os.makedirs(folder, exist_ok=True)
    for file in os.listdir(folder):
        os.remove(os.path.join(folder, file))

    for reg_id in range(1, 26):
        download_function(reg_id)

Saving region 1 in CSVs\data_region_1_2025-04-16_14-10-53.csv [OK]
Saving region 2 in CSVs\data_region_2_2025-04-16_14-10-55.csv [OK]
Saving region 3 in CSVs\data_region_3_2025-04-16_14-10-56.csv [OK]
Saving region 4 in CSVs\data_region_4_2025-04-16_14-10-57.csv [OK]
Saving region 5 in CSVs\data_region_5_2025-04-16_14-10-58.csv [OK]
Saving region 6 in CSVs\data_region_6_2025-04-16_14-10-59.csv [OK]
Saving region 7 in CSVs\data_region_7_2025-04-16_14-11-00.csv [OK]
Saving region 8 in CSVs\data_region_8_2025-04-16_14-11-01.csv [OK]
Saving region 9 in CSVs\data_region_9_2025-04-16_14-11-02.csv [OK]
Saving region 10 in CSVs\data_region_10_2025-04-16_14-11-02.csv [OK]
Saving region 11 in CSVs\data_region_11_2025-04-16_14-11-03.csv [OK]
Saving region 12 in CSVs\data_region_12_2025-04-16_14-11-04.csv [OK]
Saving region 13 in CSVs\data_region_13_2025-04-16_14-11-05.csv [OK]
Saving region 14 in CSVs\data_region_14_2025-04-16_14-11-06.csv [OK]
Saving region 15 in CSVs\data_region_15_2025-04-16_1

Зчитування та очищення даних
---

Далі нам потрібно очистити дані від html сміття та об'єднати у єдиний датафрейм.

In [3]:
def merge_cleaned_data(dir):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', '-']
    files = os.listdir(dir)
    merged_dataframe = pd.DataFrame()

    for file in files:
        try:
            dataframe = pd.read_csv(os.path.join(dir, file), header=1, names=headers)

            dataframe = dataframe[dataframe["VHI"] != -1]

            dataframe["area_id"] = file.split("_")[2]

            dataframe["area_id"] = pd.to_numeric(dataframe["area_id"], errors="coerce")

            dataframe["Year"] = dataframe["Year"].astype(str).str.replace("<tt><pre>", "", regex=False)

            dataframe = dataframe[~dataframe["Year"].str.contains('</pre></tt>', na=False)]

            dataframe["Year"] = pd.to_numeric(dataframe["Year"], errors="coerce")

            dataframe.drop('-', axis=1, inplace=True, errors="ignore")

            merged_dataframe = pd.concat([merged_dataframe, dataframe], ignore_index=True).drop_duplicates()

        except Exception as e:
            print(f"[Error] in processing file {file}: {e}")

    merged_dataframe["area_id"] = merged_dataframe["area_id"].astype(int)
    merged_dataframe = merged_dataframe.sort_values(by=["area_id", "Year", "Week"]).reset_index(drop=True)

    return merged_dataframe

dataframe = merge_cleaned_data("CSVs")
print(dataframe.head())
print(dataframe.tail())
display(dataframe)

   Year  Week    SMN     SMT    VCI    TCI    VHI  area_id
0  1982   1.0  0.053  260.31  45.01  39.46  42.23        1
1  1982   2.0  0.054  262.29  46.83  31.75  39.29        1
2  1982   3.0  0.055  263.82  48.13  27.24  37.68        1
3  1982   4.0  0.053  265.33  46.09  23.91  35.00        1
4  1982   5.0  0.050  265.66  41.46  26.65  34.06        1
       Year  Week    SMN     SMT    VCI    TCI    VHI  area_id
49445  2020  48.0  0.139  272.46  62.49  26.15  44.32       25
49446  2020  49.0  0.125  270.68  60.09  27.23  43.66       25
49447  2020  50.0  0.113  269.47  56.87  24.97  40.92       25
49448  2020  51.0  0.104  268.52  56.18  22.78  39.48       25
49449  2020  52.0  0.102  268.03  59.77  22.05  40.91       25


,Year,Week,SMN,SMT,VCI,TCI,VHI,area_id
0,1982,1.0,0.053,260.31,45.01,39.46,42.23,1
1,1982,2.0,0.054,262.29,46.83,31.75,39.29,1
2,1982,3.0,0.055,263.82,48.13,27.24,37.68,1
3,1982,4.0,0.053,265.33,46.09,23.91,35.00,1
4,1982,5.0,0.050,265.66,41.46,26.65,34.06,1
...,...,...,...,...,...,...,...,...
49445,2020,48.0,0.139,272.46,62.49,26.15,44.32,25
49446,2020,49.0,0.125,270.68,60.09,27.23,43.66,25
49447,2020,50.0,0.113,269.47,56.87,24.97,40.92,25
49448,2020,51.0,0.104,268.52,56.18,22.78,39.48,25


Зміна індексів
---

Потрібно змінити числові індекси на назви областей.

No області  та її Назва

1 Вінницька
2 Волинська

3 Дніпропетровська
4 Донецька

5 Житомирська
6 Закарпатська

7 Запорізька
8 Івано-Франківська

9 Київська
10 Кіровоградська

11 Луганська
12 Львівська

13 Миколаївська
14 Одеська

15 Полтавська
16 Рівенська

17 Сумська
18 Тернопільська

19 Харківська
20 Херсонська

21 Хмельницька
22 Черкаська

23 Чернівецька
24 Чернігівська

25 Республіка Крим

In [4]:
def change_area_id(dataframe):
    area_mapping = {
        1: 'Вінницька облать', 2: 'Волинська область', 3: 'Дніпропетровська область', 4: 'Донецька область',
        5: 'Житомирська область', 6: 'Закарпатська область', 7: 'Запорізька область', 8: 'Івано-Франківська область',
        9: 'Київська область', 10: 'Кіровоградська область', 11: 'Луганська область', 12: 'Львівська область',
        13: 'Миколаївська область', 14: 'Одеська область', 15: 'Полтавська область', 16: 'Рівненська область',
        17: 'Сумська область', 18: 'Тернопільська область', 19: 'Харківська область', 20: 'Херсонська область',
        21: 'Хмельницька область', 22: 'Черкаська область', 23: 'Чернівецька область', 24: 'Чернігівська область',
        25: 'Крим'
    }

    dataframe["area_id"] = dataframe["area_id"].replace(area_mapping)

    return dataframe



dataframe = change_area_id(dataframe)
print(dataframe.head())
print(dataframe.tail())
display(dataframe)

   Year  Week    SMN     SMT    VCI    TCI    VHI           area_id
0  1982   1.0  0.053  260.31  45.01  39.46  42.23  Вінницька облать
1  1982   2.0  0.054  262.29  46.83  31.75  39.29  Вінницька облать
2  1982   3.0  0.055  263.82  48.13  27.24  37.68  Вінницька облать
3  1982   4.0  0.053  265.33  46.09  23.91  35.00  Вінницька облать
4  1982   5.0  0.050  265.66  41.46  26.65  34.06  Вінницька облать
       Year  Week    SMN     SMT    VCI    TCI    VHI area_id
49445  2020  48.0  0.139  272.46  62.49  26.15  44.32    Крим
49446  2020  49.0  0.125  270.68  60.09  27.23  43.66    Крим
49447  2020  50.0  0.113  269.47  56.87  24.97  40.92    Крим
49448  2020  51.0  0.104  268.52  56.18  22.78  39.48    Крим
49449  2020  52.0  0.102  268.03  59.77  22.05  40.91    Крим


,Year,Week,SMN,SMT,VCI,TCI,VHI,area_id
0,1982,1.0,0.053,260.31,45.01,39.46,42.23,Вінницька облать
1,1982,2.0,0.054,262.29,46.83,31.75,39.29,Вінницька облать
2,1982,3.0,0.055,263.82,48.13,27.24,37.68,Вінницька облать
3,1982,4.0,0.053,265.33,46.09,23.91,35.00,Вінницька облать
4,1982,5.0,0.050,265.66,41.46,26.65,34.06,Вінницька облать
...,...,...,...,...,...,...,...,...
49445,2020,48.0,0.139,272.46,62.49,26.15,44.32,Крим
49446,2020,49.0,0.125,270.68,60.09,27.23,43.66,Крим
49447,2020,50.0,0.113,269.47,56.87,24.97,40.92,Крим
49448,2020,51.0,0.104,268.52,56.18,22.78,39.48,Крим


VHI ряд для області за вказаний рік
---

Реалізація функції, що виводить ряд та його екстремуми з середнім та медіанним значеннями.

In [5]:
def vhi_row(dataframe, region, year):
    
    
    filtered_subset = dataframe[(dataframe["area_id"] == region) & (dataframe["Year"] == year)]
    vhi_data = filtered_subset["VHI"]
    vhi_min = vhi_data.min()
    vhi_max = vhi_data.max()
    vhi_avg = vhi_data.mean()
    vhi_median = vhi_data.median()
    return_subset = filtered_subset[['Year', 'Week', 'VHI', 'area_id']]
    print(f"Дані VHI для {region} {year} року:")
    print(return_subset)
    print(f"Мінімальне значення: {vhi_min}")
    print(f"Максимальне значення: {vhi_max}")
    print(f"Середнє значення: {vhi_avg:.2f}")
    print(f"Медіанне значення: {vhi_median:.2f}")

reg = "Крим"
year = 2014

vhi_row(dataframe, reg, year)

Дані VHI для Крим 2014 року:
       Year  Week    VHI area_id
49086  2014   1.0  47.48    Крим
49087  2014   2.0  50.15    Крим
49088  2014   3.0  51.74    Крим
49089  2014   4.0  52.12    Крим
49090  2014   5.0  50.23    Крим
49091  2014   6.0  46.83    Крим
49092  2014   7.0  45.18    Крим
49093  2014   8.0  44.77    Крим
49094  2014   9.0  45.44    Крим
49095  2014  10.0  46.76    Крим
49096  2014  11.0  47.92    Крим
49097  2014  12.0  50.40    Крим
49098  2014  13.0  51.49    Крим
49099  2014  14.0  51.59    Крим
49100  2014  15.0  52.34    Крим
49101  2014  16.0  54.02    Крим
49102  2014  17.0  58.54    Крим
49103  2014  18.0  62.39    Крим
49104  2014  19.0  64.46    Крим
49105  2014  20.0  66.15    Крим
49106  2014  21.0  66.12    Крим
49107  2014  22.0  64.99    Крим
49108  2014  23.0  62.52    Крим
49109  2014  24.0  60.78    Крим
49110  2014  25.0  58.55    Крим
49111  2014  26.0  55.48    Крим
49112  2014  27.0  53.10    Крим
49113  2014  28.0  51.29    Крим
49114  2014  2

VHI ряд за вказаний діапазон років для вказаних областей
---



In [10]:
def vhi_row(dataframe, regions, start_year, end_year):
    

    filtered_subset = dataframe[(dataframe["area_id"].isin(regions)) & (dataframe["Year"].between(start_year, end_year))]
    return_subset = filtered_subset[['Year', 'Week', 'VHI', 'area_id']]
    print(f"Дані VHI:")
    print(return_subset)
    return return_subset

regs = ["Крим", "Донецька область"]
start = 1991
end = 2014

display(vhi_row(dataframe, regs, start, end))


Дані VHI:
       Year  Week    VHI           area_id
6391   1991   1.0  37.80  Донецька область
6392   1991   2.0  38.92  Донецька область
6393   1991   3.0  39.44  Донецька область
6394   1991   4.0  39.41  Донецька область
6395   1991   5.0  38.73  Донецька область
...     ...   ...    ...               ...
49133  2014  48.0  45.04              Крим
49134  2014  49.0  45.70              Крим
49135  2014  50.0  44.16              Крим
49136  2014  51.0  44.65              Крим
49137  2014  52.0  46.61              Крим

[2418 rows x 4 columns]


,Year,Week,VHI,area_id
6391,1991,1.0,37.80,Донецька область
6392,1991,2.0,38.92,Донецька область
6393,1991,3.0,39.44,Донецька область
6394,1991,4.0,39.41,Донецька область
6395,1991,5.0,38.73,Донецька область
...,...,...,...,...
49133,2014,48.0,45.04,Крим
49134,2014,49.0,45.70,Крим
49135,2014,50.0,44.16,Крим
49136,2014,51.0,44.65,Крим


Виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей
---

In [11]:
def extreme_drought(dataframe, percent=20):
    extreme_vhi = 15
    total_areas = dataframe["area_id"].nunique()
    extreme_level = total_areas * (percent / 100)
    drought_count_for_every_year = dataframe[dataframe['VHI'] < extreme_vhi].groupby('Year')['area_id'].nunique()

    extreme_drought_for_every_year = drought_count_for_every_year[drought_count_for_every_year > extreme_level]
    print(extreme_drought_for_every_year)
    return_dataframe = pd.DataFrame()


    for year in extreme_drought_for_every_year.index:
        concat_dataframe = dataframe[(dataframe["Year"] == year) & (dataframe["VHI"] < extreme_vhi)][["Year", "area_id", "VHI"]]
        return_dataframe = pd.concat([return_dataframe, concat_dataframe], ignore_index=True).drop_duplicates()
    
    return return_dataframe

print(extreme_drought(dataframe))

Year
2000    6
Name: area_id, dtype: int64
    Year                    area_id    VHI
0   2000           Вінницька облать  14.64
1   2000           Вінницька облать  11.82
2   2000           Вінницька облать  10.81
3   2000           Вінницька облать  10.68
4   2000           Вінницька облать  12.30
5   2000           Вінницька облать  14.24
6   2000  Івано-Франківська область  14.61
7   2000  Івано-Франківська область  11.33
8   2000  Івано-Франківська область   9.36
9   2000  Івано-Франківська область   9.45
10  2000  Івано-Франківська область   9.73
11  2000  Івано-Франківська область  11.45
12  2000  Івано-Франківська область  14.29
13  2000          Луганська область  12.51
14  2000          Луганська область  10.60
15  2000          Луганська область  11.20
16  2000          Луганська область  12.32
17  2000          Луганська область  14.65
18  2000          Львівська область  14.89
19  2000          Львівська область  12.76
20  2000          Львівська область   7.81
21  2000   